In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import os
import sys

In [42]:
file = ''
temp = sc.read_visium(file, count_file=r'GSE198353_mmtv_pymt_GEX_filtered_feature_bc_matrix.h5',load_images=True)
temp.var_names_make_unique()

In [43]:
pdata = sc.read_csv('GSE198353_mmtv_pymt_ADT_t.csv')
pdata.var_names_make_unique()

In [44]:
pdata.obsm=temp.obsm
pdata.uns = temp.uns

In [45]:
pdata

AnnData object with n_obs × n_vars = 1978 × 32
    uns: 'spatial'
    obsm: 'spatial'

In [46]:
file = ''
gdata = sc.read_visium(file, count_file=r'GSE198353_mmtv_pymt_GEX_filtered_feature_bc_matrix.h5',load_images=True)
gdata.var_names_make_unique()

## ARCH 1

Arch1 (Multi-Omic Autoencoder with Spatial Attention):
Arch1 is a multi-omic integration model that incorporates gene expression, spatial information, and protein data using an autoencoder architecture with a spatial attention mechanism. The input data consists of an Anndata gene expression matrix, spatial information obtained using scanpy.read_visium, and a protein data matrix with counts of different proteins.
Arch1 utilizes a variational autoencoder (VAE) as the core component for learning a common latent representation of the multi-omic data. It consists of a single encoder that takes the combined gene expression and protein data as input and maps it to a latent space. The VAE incorporates both the reconstruction loss and the Kullback-Leibler (KL) divergence loss to optimize the latent representation.

In addition to the VAE, Arch1 incorporates a spatial attention mechanism to capture relevant spatial features. It uses the spatial coordinates (obtained from adata.obsm and adata.uns) to highlight the spatial information within the latent space representation. The spatial attention module is designed to improve the accuracy of the latent representation by reducing the loss from the decoder.

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import scanpy as sc
import numpy as np
import scipy.sparse as sp

# Define the Variational Autoencoder
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2_mean = nn.Linear(256, latent_dim)
        self.fc2_logvar = nn.Linear(256, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 256)
        self.fc4 = nn.Linear(256, input_dim)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        mean = self.fc2_mean(x)
        logvar = self.fc2_logvar(x)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mean + eps * std
        return z

    def decode(self, z):
        z = F.relu(self.fc3(z))
        x = self.fc4(z)
        return x

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decode(z)
        return x_recon, mean, logvar

# Define the Adaptive Graph Attention Layer
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features)
        self.a = nn.Parameter(torch.zeros(size=(out_features, 2*out_features)))

    def forward(self, input, adj):
        h = self.fc(input)
        e = torch.matmul(h, self.a)
        attention = F.softmax(e, dim=1)  # Compute softmax along dimension 1
        attention = torch.transpose(attention, 0, 1)  # Transpose attention tensor
        output = torch.spmm(adj, attention.t())  # Transpose attention tensor again
        return output



# Define the "arch" module
class Arch(nn.Module):
    def __init__(self, input_dim, spatial_input_dim, latent_dim):
        super(Arch, self).__init__()

        # Autoencoder
        self.autoencoder = VAE(input_dim, latent_dim)

        # Decoder
        self.decoder = nn.Linear(latent_dim, input_dim)

        # Spatial Attention
        self.spatial_attention = GraphAttentionLayer(2, latent_dim)

    def forward(self, data, spatial_data, adj):
        # Autoencoder
        recon, mean, logvar = self.autoencoder(data)
        
        # Spatial Attention
        spatial_attention_weights = self.spatial_attention(spatial_data, adj)

        # Combined Latent Representation
        combined_latent = mean + spatial_attention_weights

        # Decoder
        recon = self.decoder(combined_latent)

        return recon, mean, logvar

    





enc of stagate

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()

        self.fc1 = nn.Linear(latent_dim, 256)
        self.fc2 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [63]:
from scipy.spatial.distance import cdist



# Extract spatial information
spatial_data = gdata.obsm['spatial']

# Set the spatial distance threshold
distance_threshold = 10.0

# Compute pairwise distances between spatial points
distances = cdist(spatial_data, spatial_data)

# Create adjacency matrix based on distance threshold
adjacency = np.where(distances <= distance_threshold, 1, 0)

# Convert adjacency matrix to a sparse matrix format
adj_sparse = sp.coo_matrix(adjacency)

# Convert adjacency matrix to a dense tensor
adj_tensor = torch.tensor(adj_sparse.toarray()).float()
adj = adj_tensor


In [68]:
import torch

# Load gene expression matrix using Anndata
data = torch.tensor(gdata.X.toarray()).float()

# Load spatial information from gdata.obsm['spatial']
spatial_data = torch.tensor(gdata.obsm['spatial']).float()


# Get the input dimensions
input_dim = data.shape[1]
spatial_input_dim = spatial_data.shape[1]

# Set the latent dimension
latent_dim = 100

# Create an instance of the "arch" module
model = Arch(input_dim, spatial_input_dim, latent_dim)

# Define the reconstruction loss
reconstruction_loss = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
data = torch.tensor(data).float()
spatial_data = torch.tensor(spatial_data).float()

# Create a DataLoader for batch processing
dataset = torch.utils.data.TensorDataset(data, spatial_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        recon, mean, logvar = model(batch[0], batch[1], adj)  # Pass the adj argument

        # Compute the reconstruction loss
        recon_loss = reconstruction_loss(recon, batch[0])

        # Compute the KL divergence loss
        kl_loss = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())

        # Compute the total loss
        loss = recon_loss + kl_loss

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")



RuntimeError: mat1 and mat2 shapes cannot be multiplied (1978x1978 and 32x200)

## ARCH 2


Arch2 (Multi-Omic Autoencoder with Separate Encoders and Spatial Attention):
Arch2 is a multi-omic integration model that learns gene expression and protein data separately using two separate encoders and maps them to a shared latent space. It also incorporates spatial information and utilizes a spatial attention mechanism to improve the latent space representations.
Arch2 consists of two encoders, one for gene expression data and another for protein data. Each encoder independently processes its respective input and maps it to a separate latent space. The encoders utilize a suitable architecture (e.g., variational autoencoder) to capture the specific characteristics of each omic dataset.

After obtaining the separate latent representations, Arch2 incorporates an adaptive graph attention layer to incorporate spatial information within the latent space. This attention layer dynamically assigns importance weights to spatial features based on their relevance. The spatial attention mechanism enables the model to focus on spatially informative regions and enhance the overall integration of multi-omic data.

Finally, Arch2 utilizes a shared decoder that takes the combined latent representation as input and reconstructs the original multi-omic data. The decoder aims to minimize the loss between the reconstructed data and the original input, improving the accuracy of the learned latent representations.



In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import scanpy as sc


# Define the Variational Autoencoder
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2_mean = nn.Linear(256, latent_dim)
        self.fc2_logvar = nn.Linear(256, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 256)
        self.fc4 = nn.Linear(256, input_dim)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        mean = self.fc2_mean(x)
        logvar = self.fc2_logvar(x)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mean + eps * std
        return z

    def decode(self, z):
        z = F.relu(self.fc3(z))
        x = self.fc4(z)
        return x

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decode(z)
        return x_recon, mean, logvar

# Define the Adaptive Graph Attention Layer
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphAttentionLayer, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.a = nn.Parameter(torch.zeros(out_features, 1))
        self.leakyrelu = nn.LeakyReLU(0.2)

    def forward(self, input, adj):
        h = self.fc(input)
        e = self.leakyrelu(torch.matmul(h, self.a))
        attention = torch.sparse_softmax(torch.transpose(adj, 0, 1))
        output = torch.spmm(attention, h)
        return output

# Define the "arch2" module
class Arch2(nn.Module):
    def __init__(self, gene_input_dim, protein_input_dim, spatial_input_dim, latent_dim):
        super(Arch2, self).__init__()

        # Gene Encoder
        self.gene_encoder = VAE(gene_input_dim, latent_dim)

        # Protein Encoder
        self.protein_encoder = VAE(protein_input_dim, latent_dim)

        # Decoder
        self.decoder = nn.Linear(latent_dim, gene_input_dim)

        # Spatial Attention
        self.spatial_attention = GraphAttentionLayer(2, latent_dim)

    def forward(self, gene_data, protein_data, spatial_data):
        # Gene Encoder
        gene_recon, gene_mean, gene_logvar = self.gene_encoder(gene_data)

        # Protein Encoder
        protein_recon, protein_mean, protein_logvar = self.protein_encoder(protein_data)

        # Spatial Attention
        spatial_attention_weights = self.spatial_attention(spatial_data, adj)

        # Combined Latent Representation
        combined_latent = gene_mean + protein_mean + spatial_attention_weights

        # Decoder
        gene_recon = self.decoder(combined_latent)

        return gene_recon, gene_mean, gene_logvar



enc of STAGATE

In [9]:
class ProteinEncoder(nn.Module):
    def __init__(self, protein_dim, latent_dim):
        super(ProteinEncoder, self).__init__()

        self.fc1 = nn.Linear(protein_dim, 256)
        self.fc2 = nn.Linear(256, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class GeneEncoder(nn.Module):
    def __init__(self, gene_dim, latent_dim):
        super(GeneEncoder, self).__init__()

        self.fc1 = nn.Linear(gene_dim, 256)
        self.fc2 = nn.Linear(256, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()

        self.fc1 = nn.Linear(latent_dim, 256)
        self.fc2 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [10]:
# Load gene expression matrix using Anndata
gene_data = ...

# Load protein data matrix
protein_data = ...

# Load spatial information using scanpy.read_visium
spatial_data = sc.read_visium(...)

# Get the input dimensions
gene_input_dim = gene_data.shape[1]
protein_input_dim = protein_data.shape[1]
spatial_input_dim = spatial_data.shape[1]

# Set the latent dimension
latent_dim = 100

# Create an instance of the "arch2" module
model = Arch2(gene_input_dim, protein_input_dim, spatial_input_dim, latent_dim)

# Define the loss function
reconstruction_loss = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
gene_data = torch.tensor(gene_data).float()
protein_data = torch.tensor(protein_data).float()
spatial_data = torch.tensor(spatial_data).float()

# Create a DataLoader for batch processing
dataset = torch.utils.data.TensorDataset(gene_data, protein_data, spatial_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        gene_recon, gene_mean, gene_logvar = model(batch[0], batch[1], batch[2])

        # Compute the reconstruction loss
        recon_loss = reconstruction_loss(gene_recon, batch[0])

        # Compute the KL divergence loss
        kl_loss = -0.5 * torch.sum(1 + gene_logvar - gene_mean.pow(2) - gene_logvar.exp())

        # Compute the total loss
        loss = recon_loss + kl_loss

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")


TypeError: expected str, bytes or os.PathLike object, not ellipsis